In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator and then re-execute this cell.')
else:
    print(gpu_info)

Thu Jun 24 06:37:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from psutil import virtual_memory

ram = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM.'.format(ram))
if ram < 20:
    print('To enable a high-RAM runtime, select the Runtime > "Change runtime type", then select High-RAM in the Runtime shape dropdown '
          'and then re-execute this cell.')
else:
    print('You are using a high-RAM runtime.')

Your runtime has 27.3 gigabytes of available RAM.
You are using a high-RAM runtime.


In [4]:
from tensorflow import config

physical_devices = config.list_physical_devices('GPU')
try:
    config.experimental.set_memory_growth(physical_devices[0], True)
except Exception as exception:
    print(exception)

In [5]:
!pip install tensorflow-addons

     |████████████████████████████████| 686kB 6.5MB/s 


In [6]:
from tensorflow import device
from tensorflow_addons import layers as new_layers
from tensorflow.keras import losses
from tensorflow.keras import models

import csv
import gc
import h5py
import imageio
import numpy as np
import os

In [7]:
town = 'Moscow'  #@param ['Berlin', 'Istanbul', 'Moscow']

model_path = '/content/gdrive/My Drive/Licenta/Traffic4Cast/All/checkpoints/auto_encoder_12/model_0.h5'
with device('gpu:0'):
    model = models.load_model(model_path)
loss = losses.MeanSquaredError()

files = '/content/gdrive/My Drive/Licenta/Traffic4Cast/{}/files/validation'.format(town)

logs = '/content/gdrive/My Drive/Licenta/Traffic4Cast/All/logs/auto_encoder_12/validation/{}_logs.csv'.format(town.lower())

movies = '/content/gdrive/My Drive/Licenta/Traffic4Cast/All/movies/validation/auto_encoder_12'

movie_names = ['ne_volume', 'ne_average_speed', 'se_volume', 'se_average_speed', 
               'sv_volume', 'sv_average_speed', 'nv_volume', 'nv_average_speed']

In [8]:
def get_file_names(files):
    return os.listdir(files)

In [9]:
def get_data(file_path):
    file = h5py.File(file_path, 'r')
    group_key = list(file.keys())[0]
    data = np.array(file[group_key][:], dtype=np.float32)  # (288, 495, 436, 9)
    file.close()
    return data

In [10]:
def get_validation_data(data):
    data = np.take(data, np.arange(8), axis=-1)  # keep only the dynamic channels
    data = np.array(np.split(data, 12))  # split in 12 batches of 12 + 12 timestamps
    return data / 255.0

In [11]:
def get_restored_data(data):
    data *= 255.0
    return data.reshape((-1, 495, 436, 8))  # (144, 495, 436, 8)

In [12]:
def get_mask(data):
    data = np.array(np.split(data, 12))  # split in 12 batches of 12 + 12 timestamps
    data = data[:, :12, :, :, :]  # keep only the inputs
    data = data.reshape(-1, 495, 436, 9)
    data = np.take(data, [1, 3, 5, 7], axis=-1)  # take only the average speed channels
    data = np.moveaxis(data, -1, 1).reshape(-1, 495, 436)  # (144 * 4, 495, 436)
    data = np.mean(data, axis=0, dtype=np.float32)
    return (data > 0.0).astype(np.uint8)

In [13]:
def create_movie(actual, prediction, mask, movies, file_path):
    actual = np.sum(actual, axis=-1, dtype=np.uint8)
    prediction = np.sum(prediction, axis=-1, dtype=np.uint8)
    prediction *= mask
    frames = np.concatenate((actual, prediction), axis=-1)
    path = os.path.join(movies, os.path.basename(file_path)[:-7] + '.gif')
    imageio.mimsave(path, [frame for frame in frames], fps=3)
    print('created movie:', path)

In [14]:
def create_movies(actual, prediction, mask, movies, file_path):
    for channel in range(8):
        first = np.squeeze(actual[:, :, :, channel])
        second = np.squeeze(prediction[:, :, :, channel])
        second *= mask
        frames = np.concatenate((first, second), axis=-1)
        frames = np.clip(frames * 3.0, 0.0, 255.0).astype('uint8')
        path = os.path.join(movies, os.path.basename(file_path)[:-6] + movie_names[channel] + '.gif')
        imageio.mimsave(path, [frame for frame in frames], fps=3)
        print('created movie:', path)

In [15]:
def get_loss(model, loss, file_path, movies):
    data = get_data(file_path)
    data = get_validation_data(data)
    actuals = data[:, 12:, :, :, :]
    predictions = np.empty(shape=(0, 12, 495, 436, 8), dtype=np.float32)
    for batch in range(0, 12, 3):
        inputs = data[batch:batch + 3, :12, :, :, :]
        actual = data[batch:batch + 3, 12:, :, :, :]
        with device('gpu:0'):
            prediction = model.predict(inputs)
            prediction = np.clip(prediction, 0.0, 1.0)
        gc.collect()
        predictions = np.concatenate((predictions, prediction), axis=0)
    actuals = np.take(actuals, [0, 1, 2, 5, 8, 11], axis=1)
    predictions = np.take(predictions, [0, 1, 2, 5, 8, 11], axis=1)
    return loss(actuals, predictions).numpy()

In [16]:
file_names = get_file_names(files)

log_file = open(logs, 'w', newline='')
log_writer = csv.writer(log_file)
log_writer.writerow(['file', 'loss'])
log_file.flush()

losses = np.zeros(shape=(len(file_names),), dtype=np.float64)

for index, file_name in enumerate(file_names):
    print('file:', index)
    result = get_loss(model, loss, os.path.join(files, file_name), movies)
    print('validation loss:', result)
    log_writer.writerow([file_name, result])
    log_file.flush()
    losses[index] = result

mean = np.mean(losses)
median = np.median(losses)
print('validation loss mean:', mean)
print('validation loss median:', median)

log_writer.writerow(['mean', mean])
log_writer.writerow(['median', median])
log_file.flush()

log_file.close()

file: 0
validation loss: 0.0015883584
file: 1
validation loss: 0.0015469088
file: 2
validation loss: 0.0016358211
file: 3
validation loss: 0.0015465353
file: 4
validation loss: 0.0015829158
file: 5
validation loss: 0.0015453165
file: 6
validation loss: 0.0015806408
file: 7
validation loss: 0.0015529131
file: 8
validation loss: 0.0015499034
file: 9
validation loss: 0.0015883358
file: 10
validation loss: 0.0014902625
file: 11
validation loss: 0.0016228467
file: 12
validation loss: 0.0014913549
file: 13
validation loss: 0.0015585296
file: 14
validation loss: 0.0015346882
file: 15
validation loss: 0.0014855281
file: 16
validation loss: 0.0015774454
file: 17
validation loss: 0.0014922542
validation loss mean: 0.0015539199165585968
validation loss median: 0.0015514082624576986


In [17]:
file_path = os.path.join(files, file_names[np.argmin(losses)])
data = get_data(file_path)
mask = get_mask(data)
data = get_validation_data(data)
actuals = data[:, 12:, :, :, :]
predictions = np.empty(shape=(0, 12, 495, 436, 8), dtype=np.float32)
for batch in range(0, 12, 3):
    inputs = data[batch:batch + 3, :12, :, :, :]
    actual = data[batch:batch + 3, 12:, :, :, :]
    with device('gpu:0'):
        prediction = model.predict(inputs)
        prediction = np.clip(prediction, 0.0, 1.0)
    gc.collect()
    predictions = np.concatenate((predictions, prediction), axis=0)
actual = get_restored_data(actuals)
prediction = get_restored_data(predictions)

In [18]:
create_movie(actual, prediction, mask, movies, file_path)

created movie: /content/gdrive/My Drive/Licenta/Traffic4Cast/All/movies/validation/auto_encoder_12/2019-11-28_moscow.gif


In [19]:
create_movies(actual, prediction, mask, movies, file_path)

created movie: /content/gdrive/My Drive/Licenta/Traffic4Cast/All/movies/validation/auto_encoder_12/2019-11-28_moscow_ne_volume.gif
created movie: /content/gdrive/My Drive/Licenta/Traffic4Cast/All/movies/validation/auto_encoder_12/2019-11-28_moscow_ne_average_speed.gif
created movie: /content/gdrive/My Drive/Licenta/Traffic4Cast/All/movies/validation/auto_encoder_12/2019-11-28_moscow_se_volume.gif
created movie: /content/gdrive/My Drive/Licenta/Traffic4Cast/All/movies/validation/auto_encoder_12/2019-11-28_moscow_se_average_speed.gif
created movie: /content/gdrive/My Drive/Licenta/Traffic4Cast/All/movies/validation/auto_encoder_12/2019-11-28_moscow_sv_volume.gif
created movie: /content/gdrive/My Drive/Licenta/Traffic4Cast/All/movies/validation/auto_encoder_12/2019-11-28_moscow_sv_average_speed.gif
created movie: /content/gdrive/My Drive/Licenta/Traffic4Cast/All/movies/validation/auto_encoder_12/2019-11-28_moscow_nv_volume.gif
created movie: /content/gdrive/My Drive/Licenta/Traffic4Cast/A